# TensorBoard 測試

In [7]:
# 刪除 log 目錄
!rd .\logs /S /Q 

In [8]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist


# 匯入 MNIST 手寫阿拉伯數字 訓練資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


# 訓練/測試資料的 X/y 維度
x_train.shape, y_train.shape,x_test.shape, y_test.shape

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
# 顏色範圍：0~255，所以，公式簡化為 x / 255
# 注意，顏色0為白色，與RGB顏色不同，(0,0,0) 為黑色。
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

In [9]:
# 建立模型
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])


In [10]:
# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define 訓練及測試的效能衡量指標(Metrics)
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [11]:
def train_step(model, optimizer, x_train, y_train):
    # 自動微分
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 計算訓練的效能衡量指標
    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    # 預測
    predictions = model(x_test)
    # 計算損失
    loss = loss_object(y_test, predictions)

    # 計算測試的效能衡量指標
    test_loss(loss)
    test_accuracy(y_test, predictions)

In [12]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 指定訓練的 log 檔名
train_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\train'
# 指定測試的 log 檔名
test_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\test'

# 開啟檔案
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [14]:
# 將訓練/測試資料轉成 Tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# 每次從 60000 筆訓練資料隨機抽出 64 筆
train_dataset = train_dataset.shuffle(60000).batch(64)
# 每次從 10000 筆測試資料隨機抽出 64 筆
test_dataset = test_dataset.batch(64)

In [15]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                                                 train_loss.result(), 
                                                 train_accuracy.result()*100,
                                                 test_loss.result(), 
                                                 test_accuracy.result()*100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 10.923014640808105, Accuracy: 32.018333435058594, Test Loss: 10.517812728881836, Test Accuracy: 34.68000030517578
Epoch 2, Loss: 9.937530517578125, Accuracy: 38.22833251953125, Test Loss: 9.53105640411377, Test Accuracy: 40.77000045776367
Epoch 3, Loss: 9.210996627807617, Accuracy: 42.731666564941406, Test Loss: 8.645798683166504, Test Accuracy: 46.18000030517578
Epoch 4, Loss: 8.332685470581055, Accuracy: 48.17500305175781, Test Loss: 7.5365800857543945, Test Accuracy: 53.1099967956543
Epoch 5, Loss: 7.85085391998291, Accuracy: 51.19333267211914, Test Loss: 7.9227752685546875, Test Accuracy: 50.790000915527344


In [16]:
# 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
%load_ext tensorboard

In [17]:
# 啟動 Tensorboard
%tensorboard --logdir logs/gradient_tape

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 15784.

## 使用瀏覽器輸入以下網址，即可觀看訓練資訊：
## http://localhost:6006/

In [29]:
!taskkill /IM "tensorboard.exe" /F
# or 
# !taskkill /F /PID 15784

成功: 處理程序 "tensorboard.exe" (PID 17852) 已經終止了。


## 寫入圖片

In [26]:
# 任意找一張圖片
img = x_train[0].numpy().reshape((-1, 28, 28, 1))
img.shape

(1, 28, 28, 1)

In [27]:
# 指定 log 檔名
logdir = ".\\logs\\train_data\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    # 將圖片寫入 log 檔
    tf.summary.image("Training data", img, step=0)

In [28]:
%tensorboard --logdir logs/train_data

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 17852.